In [1]:
import numpy as np
from sampler import *
import matplotlib.pyplot as plt
import scipy.special as sp
import scipy.stats as stats
from math import prod

np.random.seed(1)

def uniform(x):
    if x < 0:
        return 0
    if x > 1:
        return 0
    else:
        return 1

def normal(x):
    return np.exp(-x**2/2)/np.sqrt(2*np.pi)

x0 = 10
sigma = 1

def erf(x):
    if x > 0:
        return (np.exp(-(x-x0)**2/(2*sigma**2) - np.exp(-(x+x0)**2/(2*sigma**2))))/(sigma*np.sqrt(np.pi*2))/sp.erf(x0/sigma/np.sqrt(2))
    else:
        return 0

uniform_sampler = Sampler(uniform, domain=(True, 0, 1))
normal_sampler = Sampler(normal, domain=(True, -5, 5))
erf_sampler = Sampler(erf, domain=(True, 5, 15))

# Exercise 9

## a)

In [2]:
# Create data
uniform_samples = [uniform_sampler.sample(10), uniform_sampler.sample(100), uniform_sampler.sample(1000), uniform_sampler.sample(10000)]
normal_samples = [normal_sampler.sample(10), normal_sampler.sample(100), normal_sampler.sample(1000), normal_sampler.sample(10000)]
erf_samples = [erf_sampler.sample(10), erf_sampler.sample(100), erf_sampler.sample(1000), erf_sampler.sample(10000)]

# Prepare data
for samples in [uniform_samples, normal_samples, erf_samples]:
    for i in range(len(samples)):
        samples[i].set_moments()


In [3]:
# Calculating estimators for N=1000 samples
# Estimator 1
for samples in [uniform_samples, normal_samples, erf_samples]:
    print(f'Estimator 1: x = {samples[2].average:.4f}')
print()

Estimator 1: x = 0.5026
Estimator 1: x = 0.0175
Estimator 1: x = 9.9484



In [4]:
# Estimator 2
for samples in [uniform_samples, normal_samples, erf_samples]:
    x = sum(samples[2].data[:10])/10
    print(f'Estimator 2: x = {x:.4f}')


Estimator 2: x = 0.5273
Estimator 2: x = 0.0195
Estimator 2: x = 9.1660


In [5]:
# Estimator 3
for samples in [uniform_samples, normal_samples, erf_samples]:
    x = sum(samples[2].data)/(samples[2].N - 1)
    print(f'Estimator 3: x = {x:.4f}')

Estimator 3: x = 0.5031
Estimator 3: x = 0.0175
Estimator 3: x = 9.9583


In [6]:
# Estimator 4
for samples in [uniform_samples, normal_samples, erf_samples]:
    print(f'Estimator 4: x = 1.8')

Estimator 4: x = 1.8
Estimator 4: x = 1.8
Estimator 4: x = 1.8


In [7]:
# Calculating estimators for N=100 samples
# Estimator 5
for samples in [uniform_samples, normal_samples, erf_samples]:
    n = samples[2].N
    y = [xi**(1/n) for xi in samples[2].data]
    x = prod(y)
    # x = prod(samples[1].data)**(1/samples[1].N)
    print(f'Estimator 5: x = {x:.4f}')

Estimator 5: x = 0.3670
Estimator 5: x = nan
Estimator 5: x = 9.9013


/tmp/ipykernel_49052/1034821145.py:5: RuntimeWarning: invalid value encountered in double_scalars
  y = [xi**(1/n) for xi in samples[2].data]


There are few problems with this estimator: it 'breaks' for distributions near 0. There's a chance the product will become 0 because of a 'rogue' sample or negative, in which case it is not possible to take a root.

In [8]:
# Since this is a continuous distribution and finding two or more equal 
# values is basically impossible, I'll bin the data and return the highetst bin
# Estimator 6
uniform_samples[2].set_bins(0, 1, 100)
normal_samples[2].set_bins(-3, 3, 100)
erf_samples[2].set_bins(x0-3*sigma, x0+3*sigma, 100)

for samples in [uniform_samples, normal_samples, erf_samples]:
    max_hist = max(samples[2].histogram)
    i = np.where(samples[2].histogram == max_hist)[0][0]
    mode = samples[2].bins[i] 
    print(f'Estimator 6: x = {mode:.4f}')

Estimator 6: x = 0.0100
Estimator 6: x = -0.0600
Estimator 6: x = 10.0000


In [9]:
# Estimator 7
for samples in [uniform_samples, normal_samples, erf_samples]:
    x = (min(samples[2].data) + max(samples[2].data))/2
    print(f'Estimator 7: x = {x:.4f}')

Estimator 7: x = 0.4986
Estimator 7: x = -0.2555
Estimator 7: x = 9.7930


In [10]:
# Estimator 8
for samples in [uniform_samples, normal_samples, erf_samples]:
    x = sum(samples[2].data[::2])/samples[2].N*2 if samples[2].N % 2 == 0 else sum(samples[2].data[::2])/(samples[2].N-1)*2
    print(f'Estimator 8: x = {x:.4f}')

Estimator 8: x = 0.5029
Estimator 8: x = 0.0573
Estimator 8: x = 9.9662


## b)

In [11]:
# Create data
N = [10**(1+i) for i in range(5)]

uniform_samples = [uniform_sampler.sample(n) for n in N]
normal_samples = [normal_sampler.sample(n) for n in N]
erf_samples = [erf_sampler.sample(n) for n in N]

for i in range(len(N)):
    uniform_samples[i].set_moments()
    normal_samples[i].set_moments()
    erf_samples[i].set_moments()

In [12]:
# Testing consistency
#Estimator 1
print('Estimator 1')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        err += abs(samples[i].average - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')


Estimator 1
0.21947 10
0.06098 100
0.03882 1000
0.01089 10000
0.00134 100000


In [13]:
print('Estimator 2')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        x = sum(samples[i].data[:10])/10
        err += abs(x - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')

Estimator 2
0.21947 10
0.35660 100
0.19831 1000
0.41252 10000
0.33553 100000


In [14]:
print('Estimator 3')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        x = sum(samples[i].data)/(samples[i].N-1)
        err += abs(x - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')

Estimator 3
0.37038 10
0.09358 100
0.03536 1000
0.01121 10000
0.00130 100000


In [15]:
print('Estimator 4')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        x = 1.8
        err += abs(x - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')

Estimator 4
3.76667 10
3.76667 100
3.76667 1000
3.76667 10000
3.76667 100000


In [16]:
print('Estimator 5')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        n = samples[i].N
        y = [xi**(1/n) for xi in samples[i].data]
        x = prod(y)
        err += abs(x - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')

Estimator 5
nan 10
nan 100
nan 1000
nan 10000
nan 100000


/tmp/ipykernel_49052/3983979451.py:7: RuntimeWarning: invalid value encountered in double_scalars
  y = [xi**(1/n) for xi in samples[i].data]


Again, there are few problems with this estimator: it 'breaks' for distributions near 0. There's a chance the product will become 0 because of a 'rogue' sample or negative, in which case it is not possible to take a root.

In [17]:
print('Estimator 6')

for i in range(len(N)):
    uniform_samples[i].set_bins(0, 1, 100)
    normal_samples[i].set_bins(-3, 3, 100)
    erf_samples[i].set_bins(x0-3*sigma, x0+3*sigma, 100)
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        max_hist = max(samples[i].histogram)
        j = np.where(samples[i].histogram == max_hist)[0][0]
        mode = samples[i].bins[j]
        err += abs(mode-mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')

Estimator 6
0.86667 10
0.42000 100
0.34333 1000
0.14667 10000
0.08000 100000


In [18]:
print('Estimator 7')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        x = (max(samples[i].data) + min(samples[i].data))/2
        err += abs(x - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')

Estimator 7
0.23714 10
0.04840 100
0.17370 1000
0.09315 10000
0.01855 100000


In [19]:
print('Estimator 8')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        x = sum(samples[i].data[::2])/samples[i].N*2 if samples[i].N % 2 == 0 else sum(samples[i].data[::2])/(samples[i].N-1)*2
        err += abs(x - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')

Estimator 8
0.11766 10
0.02764 100
0.04458 1000
0.00782 10000
0.00244 100000


In [20]:
# Testing bias
#Estimator 1
print('Estimator 1')

for i in range(len(N)):
    err = 0
    for samples, mu in [(uniform_samples, 0.5), (normal_samples, 0), (erf_samples, 10)]:
        err += abs(samples[i].average - mu)
    err /= 3
    print(f'{err:.5f} {N[i]}')


Estimator 1
0.21947 10
0.06098 100
0.03882 1000
0.01089 10000
0.00134 100000
